### RED Events 2 TSV: Aims of this script

The aims of this script are to create events files that are compatible with bids for the RED Go/NoGo task

What we need are from the events.txt files (trials.txt have much more info but this one has the onset times we need to specify)

Required:
- onset
- duration

What we should also specify
- condition (how we have it named in the actual file)
- trial_type (go/nogo)
- emotion

Steps are as such:
- Define functions to set condition names, trial types, and emotions
- Read in events files for each subject and identify corresponding bids_id (see RED_bids_cbu_ids.csv for matching)
- In the events column, identify where blocks begin
- Extract first MRI pulse: this will be the first
- Identify first stim onset per block
- Subtract first stim onset by MRI pulse and divide by 1000 to convert to sec
- Write to tsv file into directory

In [1]:
# import modules
import os
import re
import csv, operator
import pandas as pd
import scipy as sio
import numpy as np
from decimal import *
from enum import Enum

In [3]:
# First identify directories
files_dir ='/imaging/su01/RED/GNG Task Data'
output_dir = '/imaging/su01/RED/bids'

In [4]:
# Identify subject list (red_files correspond to bids_ids order)
red_events = ["99003_events.txt", "99004_events.txt",  "99005_events.txt", "99007_events.txt",\
           "99009_events.txt", "99010_events.txt", "99011_events.txt", "99012_events.txt",\
           "99013_events.txt", "99014_events.txt", "99016_events.txt", "99018_events.txt",\
           "99019_events.txt", "99020_events.txt", "99021_events.txt", "99023_events.txt",\
           "99025_events.txt", "99026_events.txt", "99027_events.txt", "99028_events.txt",\
           "99030_events.txt", "99031_events.txt", "99034_events.txt", "99035_events.txt",\
           "99037_events.txt", "99038_events.txt", "99040_events.txt", "99044_events.txt",\
           "99047_events.txt", "99048_events.txt", "99051_events.txt", "99053_events.txt",\
           "99057_events.txt", "99058_events.txt", "99059_events.txt", "99063_events.txt",\
           "99064_events.txt", "99068_events.txt", "99070_events.txt", "99071_events.txt",\
           "99072_events.txt", "99073_events.txt", "99076_events.txt", "99078_events.txt",\
           "99080_events.txt", "99083_events.txt", "99086_events.txt", "99087_events.txt",\
           "99089_events.txt", "99090_events.txt", "99091_events.txt", "99092_events.txt",\
           "99094_events.txt", "99096_events.txt", "99097_events.txt", "99099_events.txt",\
           "99101_events.txt", "99102_events.txt", "99103_events.txt", "99106_events.txt",\
           "99107_events.txt", "99111_events.txt", "99112_events.txt", "99114_events.txt",\
           "99115_events.txt", "99118_events.txt", "99119_events.txt", "99121_events.txt",\
           "99124_events.txt", "99125_events.txt", "99129_events.txt", "99134_events.txt",\
           "99141_events.txt", "99142_events.txt", "99139_events.txt"]

bids_ids = ["sub-002", "sub-003", "sub-001", "sub-004", "sub-009", "sub-010", "sub-006", "sub-016",\
            "sub-005", "sub-008", "sub-012", "sub-011", "sub-007", "sub-014", "sub-013", "sub-019",\
            "sub-017", "sub-015", "sub-021", "sub-020", "sub-022", "sub-018", "sub-023", "sub-026",\
            "sub-028", "sub-025", "sub-027", "sub-042", "sub-024", "sub-050", "sub-051", "sub-068",\
            "sub-032", "sub-031", "sub-030", "sub-074", "sub-037", "sub-029", "sub-035", "sub-036",\
            "sub-040", "sub-041", "sub-034", "sub-038", "sub-039", "sub-047", "sub-043", "sub-046",\
            "sub-044", "sub-048", "sub-045", "sub-049", "sub-063", "sub-069", "sub-056", "sub-054",\
            "sub-058", "sub-052", "sub-057", "sub-053", "sub-070", "sub-064", "sub-055", "sub-062",\
            "sub-065", "sub-061", "sub-073", "sub-066", "sub-059", "sub-060", "sub-072", "sub-075",\
            "sub-077", "sub-076", "sub-078"]

In [5]:
# Define condition names
# Identify whether the block string variable includes the conditions ('s_n', 's_g', etc.)
# Then return the condition name only (excluding 'block onset 0..etc'
def findCondStr(blockStr):
    if re.search('s_n', blockStr):
        return 's_n'
    elif re.search('s_g', blockStr):
        return 's_g'
    elif re.search('n_n', blockStr):
        return 'n_n'
    elif re.search('n_g', blockStr):
        return 'n_g'
    elif re.search('h_n', blockStr):
        return 'h_n'
    elif re.search('h_g', blockStr):
        return 'h_g'
    elif re.search('c_n', blockStr):
        return 'c_n'
    elif re.search('c_g', blockStr):
        return 'c_g'
    else:
        return 'Invalid:' + blockStr

In [6]:
# Define the rest of the columns and have function to combine all of these
# The input variables will be resolved in the main code below
def addSummaryRecord(summary, condition, onset):
    # for the trial_type: go vs nogo
    if condition[-1] == 'n':  # if the condition str ends with 'n' this will record 'nogo' in variable g_or_ng
        g_or_ng = 'nogo'
    else:
        g_or_ng = 'go'

    # for the emotion type
    if condition[0] == 's':  # if the condition str begins with certain letter, that will determine emotion
        emotion = 'sad'
    elif condition[0] == 'n':
        emotion = 'neutral'
    elif condition[0] == 'h':
        emotion = 'happy'
    elif condition[0] == 'c':
        emotion = 'control'
    else:
        emotion = 'invalid'

    # put together this list (duration is set for 60s)
    summary.append([condition, onset, 60, g_or_ng, emotion])
    return

In [8]:
if __name__ == '__main__':
    os.chdir(files_dir)

    # for event in red_events:
    for i in range(0, len(red_events)):
        event = red_events[i]
        data = pd.read_table(event)
        block_start = mri_start = False
        summary = list()

        for index, row in data.iterrows():
            if not block_start:
                if not re.search('block', row['event']):
                    continue
                else:
                    mystr = findCondStr(row['event'])
                    block_start = True
            else:
                if not mri_start:
                    # look for MRI pulse
                    if not re.search("MRI pulse", row['event']):
                        continue
                    else:
                        mri_start = True
                        mri_pulse = float(row['time'])
                else:
                    # look for stim onset
                    if not re.search("stim onset", row['event']):
                        continue
                    else:
                        stim_onset = float(row['time'])
                        block_start = False
                        onsetTime = (stim_onset - mri_pulse) / 1000
                        addSummaryRecord(summary, mystr, onsetTime)

        # now create dataframe of all the variables
        df = pd.DataFrame(summary, columns=['conditions', 'onset', 'duration', 'trial_type', 'emotion'])
        #print(df)
        
        # save dataframe as tsv into respective output directory folder
        # set name of file with output_dir and right path
        tsvname = (output_dir + '/' + bids_ids[i] + '/' + 'func/' + bids_ids[i] + '_events.tsv')
        with open(tsvname, 'w') as tsvfile:
            tsvfile.write(df.to_csv(sep="\t", index=False))  # without row index



